In [8]:
# Content Repository Ingestion
# This notebook works in BOTH Databricks and local Jupyter
#
# PREREQUISITE: Run 0_database_setup.ipynb first to create tables!

# Install required packages
%pip install loguru pandas -q

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Restart Python (Databricks only)
try:
    dbutils.library.restartPython()
except NameError:
    print("✓ Local environment - no restart needed")

✓ Local environment - no restart needed


In [10]:
# CONFIGURATION - Using Repository Pattern (Phase 5)

from utility.environment import is_databricks
from database.config import DatabaseConfig

if is_databricks():
    # Databricks: Setup widgets
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    # Databricks paths
    DATA_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update/data_ingestion/data/1_sample_content_repo.csv"
    
    # Create database configuration
    config = DatabaseConfig(backend='databricks', catalog=CATALOG, schema=SCHEMA)
    
    print(f"📊 Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"📂 Data path: {DATA_PATH}")
else:
    # Local environment
    DATA_PATH = "data_ingestion/data/1_sample_content_repo.csv"  # Relative path
    DB_PATH = "databases/faq_update.db"  # Must match 0_database_setup.ipynb
    
    # Create database configuration
    config = DatabaseConfig(backend='sqlite', db_path=DB_PATH)
    
    print(f"💾 Local mode")
    print(f"🗄️  Database: {DB_PATH}")
    print(f"📂 Data path: {DATA_PATH}")

print()
print(f"⚙️  Backend: {config.backend}")
print()

💾 Local mode
🗄️  Database: databases/faq_update.db
📂 Data path: data_ingestion/data/1_sample_content_repo.csv

⚙️  Backend: sqlite



In [11]:
# CHECK: Verify backend connection and tables using Repository

from database.backends.factory import BackendFactory
from database.repository import ContentRepository

print("=" * 70)
print("PREREQUISITE CHECK - Repository Pattern")
print("=" * 70)
print()

# Create backend and repository
backend = BackendFactory.create_backend(config)
content_repo = ContentRepository(backend)

print(f"✓ Backend created: {backend.__class__.__name__}")
print(f"✓ Repository created: ContentRepository")
print()

# Check that we can query the database
try:
    stats = content_repo.get_file_stats()
    print(f"✓ Database connection successful")
    print(f"✓ Current file count: {stats.get('total_files', 0)}")
    print()
except Exception as e:
    print(f"❌ ERROR: Could not connect to database")
    print(f"   {str(e)}")
    print()
    print("⚠️  Please run 0_database_setup.ipynb first to create all tables.")
    raise

print("=" * 70)
print("✅ PREREQUISITE CHECK PASSED")
print("=" * 70)
print()

2025-11-02 14:42:04.602 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton
2025-11-02 14:42:04.602 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-02 14:42:04.619 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.620 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:04.621 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:04.622 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:04.624 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:04.626 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:04.628 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1

PREREQUISITE CHECK - Repository Pattern

✓ Backend created: SQLiteBackend
✓ Repository created: ContentRepository

✓ Database connection successful
✓ Current file count: 0

✅ PREREQUISITE CHECK PASSED



In [12]:
# INGEST CONTENT REPOSITORY - Using ContentRepository directly

import pandas as pd

print("=" * 70)
print("CONTENT REPOSITORY INGESTION - Repository Pattern")
print("=" * 70)
print()

# Load data
print(f"📂 Loading data from: {DATA_PATH}")

if is_databricks():
    # Databricks: Use spark to read CSV
    df = spark.read.csv(DATA_PATH, header=True, inferSchema=True).toPandas()
else:
    # Local: Use pandas
    df = pd.read_csv(DATA_PATH)

df = df.drop(columns=['content_checksum'], errors='ignore')

print(f"✓ Loaded {len(df)} records")
print()
print("📋 Sample data:")
print(df.head(3))
print()

# Get stats before
stats_before = content_repo.get_file_stats()
print("📊 Before ingestion:")
print(f"   Total files: {stats_before.get('total_files', 0)}")
print(f"   Unique files: {stats_before.get('unique_files', 0)}")
print(f"   File types: {stats_before.get('by_type', {})}")
print(f"   File statuses: {stats_before.get('by_status', {})}")
print()

# Ingest using repository directly
print("💾 Ingesting data using ContentRepository...")
try:
    result = content_repo.ingest_files_bulk(
        files_df=df,
        validate=True,
        if_exists='append'
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()

# Get stats after
stats_after = content_repo.get_file_stats()
print("📊 After ingestion:")
print(f"   Total files: {stats_after.get('total_files', 0)}")
print(f"   Unique files: {stats_after.get('unique_files', 0)}")
print(f"   File types: {stats_after.get('by_type', {})}")
print(f"   File statuses: {stats_after.get('by_status', {})}")
print()

print("=" * 70)
print("✅ INGESTION COMPLETE")
print("=" * 70)

CONTENT REPOSITORY INGESTION - Repository Pattern

📂 Loading data from: data_ingestion/data/1_sample_content_repo.csv
✓ Loaded 14 records

📋 Sample data:
             raw_file_nme raw_file_type  raw_file_version_nbr  \
0   Employee_Handbook.pdf           pdf                     1   
1  IT_Security_Policy.pdf           pdf                     1   
2  Remote_Work_Policy.pdf           pdf                     1   

                                    raw_file_path  \
0   data_ingestion/data/raw/Employee_Handbook.pdf   
1  data_ingestion/data/raw/IT_Security_Policy.pdf   
2  data_ingestion/data/raw/Remote_Work_Policy.pdf   

                        extracted_markdown_file_path  \
0  data_ingestion/data/markdown/Employee_Handbook...   
1  data_ingestion/data/markdown/IT_Security_Polic...   
2  data_ingestion/data/markdown/Remote_Work_Polic...   

                                         title_nme file_status  
0                           Employee Handbook 2024      Active  
1       IT Securi

2025-11-02 14:42:04.738 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-02 14:42:04.739 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.740 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:04.741 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:04.741 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:04.742 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:04.743 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:04.745 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.748 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 

📊 Before ingestion:
   Total files: 0
   Unique files: 0
   File types: {}
   File statuses: {}

💾 Ingesting data using ContentRepository...


2025-11-02 14:42:04.766 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:04.766 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:04.782 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 14 rows into content_repo
2025-11-02 14:42:04.783 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:04.783 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:04.784 | INFO     | database.repository.content_repository:ingest_files_bulk:712 - Successfully ingested 14 files
2025-11-02 14:42:04.784 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-02 14:42:04.784 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.784 | DEBUG    | database.repository.base:execute_

✅ SUCCESS
   Rows inserted: 14
   Message: Successfully inserted 14 rows into content_repo

📊 After ingestion:
   Total files: 14
   Unique files: 14
   File types: {'pdf': 14}
   File statuses: {'Active': 14}

✅ INGESTION COMPLETE


In [13]:
# (OPTIONAL) Verify the ingestion using Repository methods

print("=" * 70)
print("VERIFICATION - Repository Pattern")
print("=" * 70)
print()

# Use repository to list files
print("📋 Latest files in repository:")
files = content_repo.list_files(limit=10, order_by="created_dt DESC")

if files:
    for idx, file in enumerate(files, 1):
        print(f"{idx:2d}. {file['raw_file_nme']}")
        print(f"    Title: {file.get('title_nme', 'N/A')}")
        print(f"    Status: {file.get('file_status', 'N/A')}")
        print(f"    Version: {file.get('raw_file_version_nbr', 'N/A')}")
        print()
else:
    print("   No files found")

# Get overall stats
stats = content_repo.get_file_stats()
print()
print("📊 Repository Statistics:")
print(f"   Total files: {stats.get('total_files', 0)}")
print(f"   Unique files: {stats.get('unique_files', 0)}")
print()
print(f"   By Type:")
for file_type, count in stats.get('by_type', {}).items():
    print(f"      {file_type}: {count}")
print()
print(f"   By Status:")
for status, count in stats.get('by_status', {}).items():
    print(f"      {status}: {count}")
print()

if stats.get('latest_file', {}).get('name'):
    latest = stats['latest_file']
    print(f"   Latest file: {latest.get('name')} (created {latest.get('created_at')})")

print()
print("=" * 70)
print("✅ VERIFICATION COMPLETE")
print("=" * 70)

2025-11-02 14:42:04.806 | DEBUG    | database.repository.content_repository:list_files:345 - Listing files (status=None, type=None, limit=10, offset=None)
2025-11-02 14:42:04.814 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 10 rows
2025-11-02 14:42:04.815 | DEBUG    | database.repository.base:execute_query:590 - Query returned 10 rows
2025-11-02 14:42:04.815 | DEBUG    | database.repository.content_repository:list_files:382 - Found 10 files
2025-11-02 14:42:04.815 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-02 14:42:04.815 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.815 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:04.815 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:04.815 | DEBUG    | database.repository.base:exec

VERIFICATION - Repository Pattern

📋 Latest files in repository:
 1. Employee_Handbook.pdf
    Title: Employee Handbook 2024
    Status: Active
    Version: 1

 2. IT_Security_Policy.pdf
    Title: IT Security Policy - Password Requirements
    Status: Active
    Version: 1

 3. Remote_Work_Policy.pdf
    Title: Remote Work Policy - Eligibility and Guidelines
    Status: Active
    Version: 1

 4. Benefits_Guide.pdf
    Title: Employee Benefits Overview - Health Insurance
    Status: Active
    Version: 1

 5. Payroll_Processing.pdf
    Title: Payroll Processing Guide - Tax Withholding
    Status: Active
    Version: 1

 6. Code_of_Conduct.pdf
    Title: Code of Conduct - Ethics and Compliance
    Status: Active
    Version: 1

 7. Travel_Policy.pdf
    Title: Business Travel Policy - Domestic Travel
    Status: Active
    Version: 1

 8. Expense_Reimbursement.pdf
    Title: Expense Reimbursement Policy - Submission Guidelines
    Status: Active
    Version: 1

 9. Performance_Reviews.

In [14]:
# Import the database module
from database import get_backend

# Get a backend instance (auto-detects SQLite or Databricks based on environment)
backend = get_backend()

# Get dataframe from content_repo table (if it exists)
df_content_repo = backend.read_table("content_repo")

# Display the results
print("Content Repo DataFrame:")
df_content_repo

2025-11-02 14:42:04.836 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton
2025-11-02 14:42:04.840 | DEBUG    | database.backends.sqlite.backend:read_table:486 - Read 14 rows from content_repo


Content Repo DataFrame:


ud_source_file_id               raw_file_nme raw_file_type  \
0                   1      Employee_Handbook.pdf           pdf   
1                   2     IT_Security_Policy.pdf           pdf   
2                   3     Remote_Work_Policy.pdf           pdf   
3                   4         Benefits_Guide.pdf           pdf   
4                   5     Payroll_Processing.pdf           pdf   
5                   6        Code_of_Conduct.pdf           pdf   
6                   7          Travel_Policy.pdf           pdf   
7                   8  Expense_Reimbursement.pdf           pdf   
8                   9    Performance_Reviews.pdf           pdf   
9                  10         Parental_Leave.pdf           pdf   
10                 11       Health_Insurance.pdf           pdf   
11                 12        Vacation_Policy.pdf           pdf   
12                 13           Email_Policy.pdf           pdf   
13                 14         Parking_Policy.pdf           pdf   

    raw_file_version_nbr                                      raw_file_path  \
0                      1      data_ingestion/data/raw/Employee_Handbook.pdf   
1                      1     data_ingestion/data/raw/IT_Security_Policy.pdf   
2                      1     data_ingestion/data/raw/Remote_Work_Policy.pdf   
3                      1         data_ingestion/data/raw/Benefits_Guide.pdf   
4                      1     data_ingestion/data/raw/Payroll_Processing.pdf   
5                      1        data_ingestion/data/raw/Code_of_Conduct.pdf   
6                      1          data_ingestion/data/raw/Travel_Policy.pdf   
7                      1  data_ingestion/data/raw/Expense_Reimbursement.pdf   
8                      1    data_ingestion/data/raw/Performance_Reviews.pdf   
9                      1         data_ingestion/data/raw/Parental_Leave.pdf   
10                     1       data_ingestion/data/raw/Health_Insurance.pdf   
11                     1        data_ingestion/data/raw/Vacation_Policy.pdf   
12                     1           data_ingestion/data/raw/Email_Policy.pdf   
13                     1         data_ingestion/data/raw/Parking_Policy.pdf   

                         extracted_markdown_file_path  \
0   data_ingestion/data/markdown/Employee_Handbook...   
1   data_ingestion/data/markdown/IT_Security_Polic...   
2   data_ingestion/data/markdown/Remote_Work_Polic...   
3   data_ingestion/data/markdown/Benefits_Guide_v1.md   
4   data_ingestion/data/markdown/Payroll_Processin...   
5   data_ingestion/data/markdown/Code_of_Conduct_v...   
6    data_ingestion/data/markdown/Travel_Policy_v1.md   
7   data_ingestion/data/markdown/Expense_Reimburse...   
8   data_ingestion/data/markdown/Performance_Revie...   
9   data_ingestion/data/markdown/Parental_Leave_v1.md   
10  data_ingestion/data/markdown/Health_Insurance_...   
11  data_ingestion/data/markdown/Vacation_Policy_v...   
12    data_ingestion/data/markdown/Email_Policy_v1.md   
13  data_ingestion/data/markdown/Parking_Policy_v1.md   

                                            title_nme file_status  \
0                              Employee Handbook 2024      Active   
1          IT Security Policy - Password Requirements      Active   
2     Remote Work Policy - Eligibility and Guidelines      Active   
3       Employee Benefits Overview - Health Insurance      Active   
4          Payroll Processing Guide - Tax Withholding      Active   
5             Code of Conduct - Ethics and Compliance      Active   
6            Business Travel Policy - Domestic Travel      Active   
7   Expense Reimbursement Policy - Submission Guid...      Active   
8                   Annual Performance Review Process      Active   
9    Parental Leave Policy - Eligibility and Duration      Active   
10          Health Insurance Enrollment - Three Plans      Active   
11                    Vacation Policy - Accrual Based      Active   
12                                 Email Usage Policy      Active   
13   